In [ ]:
!pip install -U sentence-transformers
import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns
import altair as alt

In [ ]:
products = pd.read_csv("/content/Produktkatalog.csv", error_bad_lines=False)

In [ ]:
products['KEYWORD'] = products['KEYWORD_LIST'].apply(lambda x: 'Y' if 'LED' in x else 'N')

In [ ]:
products.head()

In [ ]:
products.shape

In [ ]:
# Define a list with sentences (1k - 100k sentences)
train_sentences = products['DESCRIPTTION_LONG']

In [ ]:
# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

In [ ]:
# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-german-cased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

In [ ]:
# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

In [ ]:
model

In [ ]:
model.save('output/productcat-tsdae-model')

In [ ]:
model.encode(train_sentences[42])

In [ ]:
embeddings = model.encode(train_sentences)

In [ ]:
embeddings

In [ ]:
pca = PCA(n_components=2)
pca.fit(embeddings)
embeddings_pcs = pca.transform(embeddings)

In [ ]:
embeddings_pcs

In [ ]:
embeddings_pcs_df = pd.DataFrame(embeddings_pcs)

In [ ]:
products['PC1'] = embeddings_pcs_df[0]
products['PC2'] = embeddings_pcs_df[1]

In [ ]:
embeddings_tsne = TSNE(n_components=2, perplexity=30, init='pca', n_jobs=-1).fit_transform(embeddings)

In [ ]:
embeddings_tsne

In [ ]:
embeddings_tsne_df = pd.DataFrame(embeddings_tsne)

In [ ]:
products['TSNE1'] = embeddings_tsne_df[0]
products['TSNE2'] = embeddings_tsne_df[1]

In [ ]:
products.head()

In [ ]:
alt.Chart(products.sample(5000)).mark_circle(size=60).encode(
    x='PC1',
    y='PC2',
    color='KEYWORD',
    tooltip=['DESCRIPTTION_LONG']
).configure_mark(
    opacity=0.3,
    color='red'
).interactive()